As we saw in the lecture, using CNN layers can generate much larger code. What is not immediately obvious is that a lot of the code is shared and could be reused. Wrapping the layers with functions as we showed in the lecture helped, but there is more available. 

Most of the shared code is used for initialization, activations functions, etc. TensorFlow contains an interface called `tf.layers` that will help share a lot more of the code. 

This example uses the same basic structure as the last one, but now uses `tf.layers` for the CNN layers. In this notebook we will see how to use `tf.layers` with our standard process. In the next notebook we will see how to use the estimator interface.


In [13]:
# cell 1
# notebook version 1.2

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

print ('cell finished')


cell finished


The first cell reads the data sets as we have been doing all along.

In [14]:
# cell 2

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

print ('cell finished')


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
cell finished


Now we define the input variable to accept digit input pixel values, and the expected result from the model. These look very familiar as we have used the same in previous models.

This time we are adding a new placeholder to control when dropout is performed. We only want dropout to occur while we are training. The drop placeholder will be set to True while training, but False when we are checking accuracy. 

In [1]:
# cell 3

x = tf.placeholder(tf.float32, [None,784])
y_ = tf.placeholder(tf.float32, [None, 10])
drop = tf.placeholder(tf.bool)
# Dropout: trains some weights during one time, trains other weights during another time.
# Technique that avoids overfitting (aka when system sees something unfamiliar, then it will behave randomly). 

print ('cell finished')


NameError: name 'tf' is not defined

This cell shows tf.layers in action. We have defined all of the layers in a single function called cnnNet. 

This function accepts a single input 2D image and a mode input to control dropout. The dropout mode will determine whether dropout is applied or not. 

The first convolutional layer accepts the input image and generates output conv1, which is the input to the next layer. The next layer accepts conv1 and generates a new output applied to the next layer. This occurs successively until all of the layers have been defined.  

Notice that each layer has a number of parameters that allow specification of the activation type, number of filters, kernel size, etc. Instead of being defined separately, these parameters allow the specification per layer.  

In [4]:
# cell 4

def cnnNet(images, mode):
    conv1 = tf.layers.conv2d(
        inputs=images,
        filters=32,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)

    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu)
    
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2],strides=2)
    pool2flat = tf.reshape(pool2, [-1, 7*7*64])
    # What does this do?
    
    dense = tf.layers.dense(inputs=pool2flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense,
      rate=0.4, # amount of weights that are dropped out, in this case, 40%
      training=mode)
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    return logits


print ('cell finished')


cell finished


Convolutional layers need 2D images as input, but our input data currently is a 784 x 1 vector. The following cell reshapes the 784x1 input vector to a 28x28 image to send to the layers above. 

In [5]:
# cell 5

x_image = tf.reshape(x, [-1,28,28,1])
# Reshapes to a 28x28 image

print ('cell finished')


cell finished


The following cell passes the 2D image to the cnnNet function to generate the categorization. It is trivial to add another layer, change a layer size, or other operations. Just make the changes in the cnnNet function definition and it will be reflected in the call below. 

In [6]:
# cell 6

y_conv = cnnNet(x_image, drop) # calls the layer function

print ('cell finished')

cell finished


All that is needed to complete the computational graph for this model is the non-linearity and loss function for the final layer. Just like previous models this will be done using a softmax. Cross entropy on the logits between the expected results and the predicted results from the model will complete the loss calculation. All that is left is to specify an optimizer. 


In [7]:
# cell 7

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

print ('cell finished')


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

cell finished


Instead of using the standard gradient descent optimizer as in the past, this model will use the AdamOptimizer. 

In [8]:
# cell 8

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

print ('cell finished')


cell finished


As in previous models the accuracy of the model is computed using the following:

In [9]:
# cell 9

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print ('cell finished')


cell finished


To run the model a session must be created, and variables initialized

In [10]:
# cell 10

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

print ('cell finished')


cell finished


Finally the training process is run. Notice that the number of steps run is much larger than previous, and the learning rate is much smaller. These numbers are based on experience. 

During training, every 100 steps the current accuracy is printed so that progress can be monitored. Notice every call to the session requires a value for the drop placeholder and the only time that drop is True is during training. 

In [11]:
# cell 11

for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], drop:False})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], drop:True})

print ('cell finished')

step 0, training accuracy 0.08
step 100, training accuracy 0.96
step 200, training accuracy 0.88
step 300, training accuracy 0.86
step 400, training accuracy 0.98
step 500, training accuracy 0.96
step 600, training accuracy 1
step 700, training accuracy 0.94
step 800, training accuracy 0.96
step 900, training accuracy 0.92
step 1000, training accuracy 1
step 1100, training accuracy 1
step 1200, training accuracy 1
step 1300, training accuracy 1
step 1400, training accuracy 1
step 1500, training accuracy 0.96
step 1600, training accuracy 0.98
step 1700, training accuracy 0.98
step 1800, training accuracy 0.96
step 1900, training accuracy 1
step 2000, training accuracy 1
step 2100, training accuracy 0.98
step 2200, training accuracy 1
step 2300, training accuracy 0.96
step 2400, training accuracy 1
step 2500, training accuracy 1
step 2600, training accuracy 0.98
step 2700, training accuracy 1
step 2800, training accuracy 0.98
step 2900, training accuracy 1
step 3000, training accuracy 0.

After training has completed the final accuracy number is calculated on the test images. 

In [12]:
# cell 12

print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, drop:False}))

print ('cell finished')

test accuracy 0.9938
cell finished


When working with images even a less than 1 percent difference can make a big difference in the utility of the classifier. Notice that the accuracy achieved with the convolutional model is significantly better than even the deep network from the last exercise. 

**IMPORTANT: When you are finished make sure you go to the Jupyter notebook “File” menu above and select “Close and halt”. This will shutdown this notebook and take you back to the Jupyter Notebook Home tab.**